<hr>

# Simulasi

Terkadang, untuk mendefinisikan sebuah ruang sampel menjadi sulit atau bahkan tidak memungkinkan. Misal, ruang sampelnya tidak terhingga, atau ruang sampelnya sangat besar dan rumit. Dalam kasus tersebut, akan lebih mudah jika kita menulis program untuk mensimulasikan hasilnya. *Random sampling* dari simulasi seperti itu biasanya dapat memberikan estimasi yang akurat terhadap nilai probabilitasnya.

# Mensimulasikan Monopoli

![Mr. Monopoly](http://buckwolf.org/a.abcnews.com/images/Entertainment/ho_hop_go_050111_t.jpg)<center>[Mr. Monopoly](https://en.wikipedia.org/wiki/Rich_Uncle_Pennybags)<br>1940&mdash;

Permasalahan berikut diadaptasi dari [problem 84](https://projecteuler.net/problem=84) [Project Euler](https://projecteuler.net) yang menanyakan probabilitas dari seorang pemain bisa berhenti di setiap kotak pada game. Untuk menjawab permasalahan ini kita perlu memasukkan faktir seperti lemparan dadu, kartu, dan masuk kedalam penjara (dari kotak "go to jail", dari kartu, atau melempar angka kembar tiga kali berturut-turut). Kita tidak perlu mempertimbangkan pembelian properti atau jumlah uang karena kejadian tersebut tidak mengubah lokasi pemain.
    
Permainan monoppoli dapat berjalan terus menerus, sehingga ruang sampelnya tidak terbatas. Bahkan jika kita membatasi ruang sampel sampai 1000 pelemparan, akan ada $21^{1000}$ lemparan dadu, belum lagi ditambah dengan kejadian karena efek kartu. Sehingga tidak memungkinkan untuk secara eksplisit merepresentasikan ruang sampel. Untuk menyelesaikan permasalahan ini terdapat beberapa cara seperti menggunakan representasi [Markov Decision Problem] (https://faculty.math.illinois.edu/~bishop/monopoly.pdf) dan menyelesaikannya. 

Namun, cara yang paling sederhana adalah melakukan simulasi dan menjalankannya sampai, misal, satu juta lemparan dadu. Berikut adalah code nya:










In [1]:
from collections import deque as Deck # a Deck of community chest or chance cards
import random

# The Monopoly board, as specified by https://projecteuler.net/problem=84
(GO,   A1, CC1, A2,  T1, R1, B1,  CH1, B2, B3,
 JAIL, C1, U1,  C2,  C3, R2, D1,  CC2, D2, D3, 
 FP,   E1, CH2, E2,  E3, R3, F1,  F2,  U2, F3, 
 G2J,  G1, G2,  CC3, G3, R4, CH3, H1,  T2, H2) = board = range(40)

# A card is either a square, a set of squares meaning advance to the nearest, 
# a -3 to go back 3 spaces, or None meaning no change to location.
CC_deck = Deck([GO, JAIL] +  14 * [None])
CH_deck = Deck([GO, JAIL, C1, E3, H2, R1, -3, {U1, U2}] 
               + 2 * [{R1, R2, R3, R4}] + 6 * [None])

def monopoly(rolls):
    """Simulate given number of dice rolls of a Monopoly game, 
    and return the counts of how often each square is visited."""
    counts = [0] * len(board)
    doubles = 0 # Number of consecutive doubles rolled
    random.shuffle(CC_deck)
    random.shuffle(CH_deck)
    goto(GO)
    for _ in range(rolls):
        d1, d2 = random.randint(1, 6), random.randint(1, 6)
        doubles = (doubles + 1 if d1 == d2 else 0)
        goto(here + d1 + d2)
        if here == G2J or doubles == 3:
            goto(JAIL)
            doubles = 0
        elif here in (CC1, CC2, CC3):
            do_card(CC_deck)
        elif here in (CH1, CH2, CH3):
            do_card(CH_deck)
        counts[here] += 1
    return counts

def goto(square):
    "Update 'here' to be this square (and handle passing GO)."
    global here
    here = square % len(board)        

def do_card(deck):
    "Take the top card from deck and do what it says."
    card = deck.popleft()       # The top card
    deck.append(card)           # Move top card to bottom of deck
    if card == None:            # Don't move
        pass
    elif card == -3:            # Go back 3 spaces
        goto(here - 3)
    elif isinstance(card, set): # Advance to next railroad or utility
        next1 = min({place for place in card if place > here} or card)
        goto(next1)
    else:                       # Go to destination named on card
        goto(card)


Kita lakukan simulasi untuk satu juta kali lemparan dadu:



In [2]:
counts = monopoly(10**6)


Dan print nama dari tiap kotak dan presentasenya:

In [3]:
property_names = """
 GO,   A1, CC1, A2,  T1, R1, B1,  CH1, B2, B3,
 JAIL, C1, U1,  C2,  C3, R2, D1,  CC2, D2, D3, 
 FP,   E1, CH2, E2,  E3, R3, F1,  F2,  U2, F3, 
 G2J,  G1, G2,  CC3, G3, R4, CH3, H1,  T2, H2""".replace(',', ' ').split()

for (c, n) in sorted(zip(counts, property_names), reverse=True):
    print('{:4} {:.2%}'.format(n, c / sum(counts)))

JAIL 6.22%
E3   3.20%
GO   3.12%
R3   3.08%
D3   3.07%
R1   2.99%
D2   2.94%
R2   2.92%
FP   2.88%
E1   2.85%
U2   2.81%
D1   2.79%
E2   2.73%
F1   2.71%
C1   2.70%
G1   2.69%
F2   2.66%
H2   2.64%
G2   2.62%
F3   2.59%
U1   2.59%
CC2  2.58%
G3   2.51%
C3   2.46%
R4   2.42%
CC3  2.39%
C2   2.38%
T1   2.31%
B2   2.31%
B3   2.28%
B1   2.27%
T2   2.19%
H1   2.17%
A2   2.16%
A1   2.11%
CC1  1.87%
CH2  1.06%
CH1  0.87%
CH3  0.85%
G2J  0.00%


Terdapat satu kotak yang jauh diatas rata-rata: `JAIL`, yang berada pada angka 6%. Dan ada 4 kotak yang cukup jauh dibawah rata-rata: 3 kotak *Chances*, `CH1`, `CH2`, dan `CH3`, pada sekitar angka 1% (Karena 10 dari 16 kartu *Chance* memindahkan posisi pemain dari kotak tersebut), dan kotak "Go to Jail", yang memiliki frekuensi 0 karena kita tidak dapat mengakhiri putaran disana (otomatis terlempar ke penjara). Kotak lainnya adalah sekitar 2% sampai 3%, yang dimana sesuai dengan perkiraan kita, karena 100% / 40 = 2.5%.